In [1]:
#Team 1 Joint Notebook for Project 3


In [2]:
#pip install dash

In [3]:
#import dependencies 
from dash import Dash, dcc, html, Input, Output, ctx
import pandas as pd
import plotly.express as px
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
import yfinance as yf
import time

In [4]:
#Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["Project3DB"]  # Database name
collectionBiden = db["Biden"]  # Biden Collection Name
collectionTrump = db["Trump"] # Trump collection Name

In [5]:
#Verify collections
print(db.list_collection_names())

['Biden', 'Trump']


In [6]:
#attempted to query the datbase to create a list for AAPL stock values
AAPLquery = {'Ticker':{'$eq': 'AAPL'}}
fields = {'Stock Data.Date', 'Stock Data.Close'}

sort = [('Date', 1)]

resultsBidenAAPL = list(collectionBiden.find(AAPLquery, fields).sort(sort))
#pprint(resultsBidenAAPL)
BidenAAPLdf = pd.DataFrame(resultsBidenAAPL)
BidenAAPLdf.head()

,_id,Stock Data
0,67acdfba21cb5534fcf06028,"[{'Date': '2021-01-20', 'Close': 128.964416503..."


In [7]:
#calling on Yahoo finance for top 5 market cap us companies
dataTrumpUS = yf.download("AAPL NVDA MSFT AMZN GOOG", start="2017-01-20",
                    end="2017-12-31", group_by='tickers')

dataTrumpUS


[*********************100%***********************]  1 of 5 completed

5 Failed downloads:
['AAPL', 'GOOG', 'MSFT', 'AMZN', 'NVDA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Empty DataFrame
Columns: [(AAPL, Open), (AAPL, High), (AAPL, Low), (AAPL, Close), (AAPL, Adj Close), (AAPL, Volume), (GOOG, Open), (GOOG, High), (GOOG, Low), (GOOG, Close), (GOOG, Adj Close), (GOOG, Volume), (MSFT, Open), (MSFT, High), (MSFT, Low), (MSFT, Close), (MSFT, Adj Close), (MSFT, Volume), (AMZN, Open), (AMZN, High), (AMZN, Low), (AMZN, Close), (AMZN, Adj Close), (AMZN, Volume), (NVDA, Open), (NVDA, High), (NVDA, Low), (NVDA, Close), (NVDA, Adj Close), (NVDA, Volume)]
Index: []

[0 rows x 30 columns]

[**********************80%*************          ]  4 of 5 completed

In [8]:
#calling on Yahoo finance for top 5 market cap non-US companies
dataTrumpOUS = yf.download("TSM SSNLF ASML TCEHY BABA", start="2017-01-20",
                    end="2017-12-31", group_by='tickers')
dataTrumpOUS

[*********************100%***********************]  5 of 5 completed**********]  8 of 5 completed

5 Failed downloads:
['ASML', 'SSNLF', 'BABA', 'TCEHY', 'TSM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Empty DataFrame
Columns: [(ASML, Open), (ASML, High), (ASML, Low), (ASML, Close), (ASML, Adj Close), (ASML, Volume), (SSNLF, Open), (SSNLF, High), (SSNLF, Low), (SSNLF, Close), (SSNLF, Adj Close), (SSNLF, Volume), (BABA, Open), (BABA, High), (BABA, Low), (BABA, Close), (BABA, Adj Close), (BABA, Volume), (TCEHY, Open), (TCEHY, High), (TCEHY, Low), (TCEHY, Close), (TCEHY, Adj Close), (TCEHY, Volume), (TSM, Open), (TSM, High), (TSM, Low), (TSM, Close), (TSM, Adj Close), (TSM, Volume)]
Index: []

[0 rows x 30 columns]

In [10]:
tickers_data = {}
tickers_list = ["AAPL", "NVDA", "MSFT", "AMZN", "GOOG", "TSM", "SAP", "ASML", "TCEHY", "BABA"]
for ticker in tickers_list:
    ticker_object = yf.Ticker(ticker)

    #convert info() output from dictionary to dataframe
    temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
    temp.reset_index(inplace=True)
    temp.columns = ["Attribute", "Recent"]
    
    # add (ticker, dataframe) to main dictionary
    tickers_data[ticker] = temp

tickers_data

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [ ]:
combined_data = pd.concat(tickers_data)
combined_data = combined_data.reset_index()
combined_data

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt


trump_file_path = "Data/Trump.json"
biden_file_path = "Data/Biden.json"

with open(trump_file_path, "r") as f:
    trump_data = json.load(f)

with open(biden_file_path, "r") as f:
    biden_data = json.load(f)

trump_companies = list(trump_data[0].keys())[:20]
biden_companies = list(biden_data[0].keys())[:20]


def extract_stock_data(data, companies):
    stock_dict = {}
    for company in companies:
        stock_data = data[0][company]["Stock Data"]
        dates = [entry["Date"] for entry in stock_data]
        prices = [entry["Close"] for entry in stock_data]
        stock_dict[company] = pd.Series(prices, index=pd.to_datetime(dates))
    
    return pd.DataFrame(stock_dict)


trump_df = extract_stock_data(trump_data, trump_companies)
biden_df = extract_stock_data(biden_data, biden_companies)


In [14]:
trump_df

,AAPL,NVDA,MSFT,AMZN,GOOG,META,TSLA,AVGO,ORCL,CRM,TSM,SSNLF,ASML,TCEHY,BABA,SAP,SONY,INFY,BIDU
2017-01-20,27.752829,2.562162,56.859623,40.416500,40.106892,126.556564,16.315332,14.964442,35.203362,75.607697,24.353594,9.217007e+29,112.693504,23.381807,92.769615,78.485817,5.187129,5.765440,173.440002
2017-01-23,27.771336,2.588767,57.059002,40.894001,40.818832,128.439377,16.594667,14.958176,35.035606,75.736984,24.564167,9.217007e+29,112.804283,23.519619,95.039131,78.398033,5.175224,5.657038,175.970001
2017-01-24,27.745893,2.643946,57.566528,41.122002,41.046021,128.877686,16.974001,15.486806,35.406441,76.681702,24.961018,1.032305e+30,113.155167,23.666618,97.955681,78.687744,5.185429,5.669082,176.050003
2017-01-25,28.187622,2.655277,57.711529,41.826000,41.633907,130.979660,16.964666,15.857235,35.450588,78.034142,25.357862,1.032305e+30,113.367531,23.997360,100.495590,80.083626,5.236450,5.620903,176.610001
2017-01-26,28.201508,2.701097,58.246227,41.957500,41.458534,132.274719,16.834000,15.989583,35.432930,77.517036,25.195887,1.032305e+30,111.844017,24.355667,99.230461,79.732468,5.292573,5.536589,174.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22,41.126072,4.828984,79.225853,58.417999,52.816231,176.525681,21.680000,20.949722,42.321541,102.059952,33.141991,-8.130178e+28,163.393707,48.089451,170.251465,100.124916,7.910932,6.651383,239.050003
2017-12-26,40.082703,4.882646,79.123940,58.838001,52.647835,175.320282,21.152666,20.610340,42.384083,101.970451,32.573307,-8.130178e+28,161.407471,47.932892,166.427109,99.929253,7.829537,6.696827,237.830002
2017-12-27,40.089752,4.875969,79.411163,59.112999,52.280651,176.944077,20.775999,20.691792,42.339405,102.079842,32.682045,9.644539e+27,162.293335,47.454021,167.045197,100.587379,7.838196,6.684434,237.710007
2017-12-28,40.202553,4.881659,79.420418,59.305000,52.219376,177.242935,21.024000,20.795603,42.464516,102.219063,33.233978,9.644539e+27,162.442551,47.794762,166.398148,99.964844,7.843392,6.676172,234.740005


In [16]:
trumpChange = trump_df.iloc[-1] - trump_df.iloc[0]
trumpChange

AAPL     1.201498e+01
NVDA     2.223049e+00
MSFT     2.239401e+01
AMZN     1.805700e+01
GOOG     1.202579e+01
META     4.923195e+01
TSLA     4.441334e+00
AVGO     5.550076e+00
ORCL     7.046692e+00
CRM      2.605448e+01
TSM      8.805120e+00
SSNLF   -9.120561e+29
ASML     4.939472e+01
TCEHY    2.443137e+01
BABA     7.375407e+01
SAP      2.144344e+01
SONY     2.597381e+00
INFY     9.355187e-01
BIDU     6.077000e+01
dtype: float64

In [18]:
bidenChange = biden_df.iloc[-1] - biden_df.iloc[0]
bidenChange

AAPL     4.617102e+01
NVDA     1.620496e+01
MSFT     1.138911e+02
AMZN     5.475494e+00
GOOG     5.147256e+01
META     7.658740e+01
TSLA     7.329666e+01
AVGO     2.023792e+01
ORCL     2.590841e+01
CRM      3.171288e+01
TSM     -7.735382e+00
SSNLF   -2.219949e+15
ASML     2.461951e+02
TCEHY   -2.664162e+01
BABA    -1.376189e+02
SAP      1.399747e+01
SONY     4.654470e+00
SOBKY   -6.220007e-01
INFY     6.793844e+00
BIDU    -1.102900e+02
dtype: float64